In [1]:
import xarray as xr
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline
xr.__version__

'0.11.1+65.g627a881'

In [2]:
import zarr
zarr.__version__

'2.2.1.dev146'

- don't forget to do some port forwarding from the machine your browser is running on:
- ssh -L 8787:localhost:8787 naomi@mary
- (and DASK DASHBOARD URL = http://127.0.0.1:8787/status)
            

In [3]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:33197 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 40 Memory: 134.90 GB


In [4]:
# From Ryan
def set_bnds_as_coords(ds):
    new_coords_vars = [var for var in ds.data_vars if 'bnds' in var or 'bounds' in var]
    ds = ds.set_coords(new_coords_vars)
    return ds

In [5]:
%%time
ncpath = '/dm5/naomi/CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/uo/gn/v20181129/uo_Omon_BCC-ESM1_historical_r1i1p1f1_gn_185001-185912.nc'
ds = xr.open_dataset(ncpath)

CPU times: user 48.5 ms, sys: 12 ms, total: 60.5 ms
Wall time: 180 ms


In [39]:
#%%time
ncfile1 = '/dm5/naomi/CMIP6mon/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/uo/gn/v20181129/uo_Omon_BCC-ESM1_historical_r1i1p1f1_gn_185001-185912.nc'
ds1 = xr.open_mfdataset(ncfile1, decode_times=False, preprocess=set_bnds_as_coords)
ncfile2 = '/dm5/naomi/CMIP6mon/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/uo/gn/v20181129/uo_Omon_BCC-ESM1_historical_r1i1p1f1_gn_186001-186912.nc'
ds2 = xr.open_mfdataset(ncfile2, decode_times=False, preprocess=set_bnds_as_coords)

In [47]:
print(ds1.time.attrs['units'],' and ',ds2.time.attrs['units'])

days since 1850-1-1  and  days since 1860-1-1


In [54]:
ds = xr.open_mfdataset([ncfile1,ncfile2], decode_times=True, preprocess=set_bnds_as_coords)
ds.coords

Coordinates:
  * lev        (lev) float64 5.0 15.0 25.0 ... 4.289e+03 4.807e+03 5.334e+03
    lev_bnds   (lev, bnds) float64 0.0 10.0 10.0 ... 5.071e+03 5.071e+03 5.6e+03
  * lat        (lat) float64 -81.0 -80.0 -79.0 -78.0 ... 87.0 88.0 89.0 90.0
    lat_bnds   (lat, bnds) float64 -81.0 -80.5 -80.5 -79.5 ... 89.5 89.5 90.0
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    lon_bnds   (lon, bnds) float64 -0.5 0.5 0.5 1.5 ... 357.5 358.5 358.5 359.5
    latitude   (lat, lon) float32 -81.0 -81.0 -81.0 ... 88.4463 88.2237 88.00084
    longitude  (lat, lon) float32 0.0 1.0 2.0 3.0 4.0 ... 80.0 80.0 80.0 80.0
  * time       (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<shape=(240, 2), chunksize=(120, 2)>

In [55]:
ds.time_bnds.load()
ds.coords

Coordinates:
  * lev        (lev) float64 5.0 15.0 25.0 ... 4.289e+03 4.807e+03 5.334e+03
    lev_bnds   (lev, bnds) float64 0.0 10.0 10.0 ... 5.071e+03 5.071e+03 5.6e+03
  * lat        (lat) float64 -81.0 -80.0 -79.0 -78.0 ... 87.0 88.0 89.0 90.0
    lat_bnds   (lat, bnds) float64 -81.0 -80.5 -80.5 -79.5 ... 89.5 89.5 90.0
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    lon_bnds   (lon, bnds) float64 -0.5 0.5 0.5 1.5 ... 357.5 358.5 358.5 359.5
    latitude   (lat, lon) float32 -81.0 -81.0 -81.0 ... 88.4463 88.2237 88.00084
    longitude  (lat, lon) float32 0.0 1.0 2.0 3.0 4.0 ... 80.0 80.0 80.0 80.0
  * time       (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bnds  (time, bnds) object 1850-01-01 00:00:00 ... 1870-01-01 00:00:00

In [56]:
ds.chunk(chunks={'time' : 'auto'})

NotImplementedError: Can not use auto rechunking with object dtype. We are unable to estimate the size in bytes of object data

In [7]:
#! rm -rf /dm4/naomi/zarr-test-Omon

In [8]:
%%time 
s='/'
zarrdir = '/dm4/naomi/zarr-test-Omon'
#variables = ['thetao','so','uo','vo','wo']
variables = ['uo']
models = glob('/dm5/naomi/CMIP6/CMIP/*/*')

for model in models:
    if 'MIROC' in model:
        continue
    print(model.split(s)[-1],'\n')
    variants = glob(model+'/historical/*')
    
    for variant in variants:
        print(variant.split(s)[-1])
        
        for var in variables:
            vpath = variant+'/Omon/'+var+'/gn/'
            versions = glob(vpath+'v*')
            version=versions[-1].split(s)[-1]
            #print(version)
            zpath = zarrdir+variant[16:]+s+version+s+var
            ncpath = versions[-1]
            print('ncpath is: ',ncpath,'\n')
            ds = xr.open_mfdataset(ncpath+'/*.nc', preprocess=set_bnds_as_coords, decode_times=False, concat_dim=['time'], chunks={'time' : 'auto'}, parallel=True)
            #ds = xr.open_mfdataset(ncpath+'/*.nc', preprocess=set_bnds_as_coords, parallel=True)
            print('zpath is: ',zpath,'\n')
            # encoding = {dvar: {'compressor': zarr.Zstd(level=3)} for dvar in ds.data_vars}
            ds.to_zarr(zpath,consolidated=True) # encoding=encoding)

BCC-ESM1 

r1i1p1f1
ncpath is:  /dm5/naomi/CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/uo/gn/v20181129 

zpath is:  /dm4/naomi/zarr-test-Omon/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/v20181129/uo 

r3i1p1f1
ncpath is:  /dm5/naomi/CMIP6/CMIP/BCC/BCC-ESM1/historical/r3i1p1f1/Omon/uo/gn/v20181218 

zpath is:  /dm4/naomi/zarr-test-Omon/CMIP/BCC/BCC-ESM1/historical/r3i1p1f1/v20181218/uo 

r2i1p1f1
ncpath is:  /dm5/naomi/CMIP6/CMIP/BCC/BCC-ESM1/historical/r2i1p1f1/Omon/uo/gn/v20181218 

zpath is:  /dm4/naomi/zarr-test-Omon/CMIP/BCC/BCC-ESM1/historical/r2i1p1f1/v20181218/uo 

BCC-CSM2-MR 

r1i1p1f1
ncpath is:  /dm5/naomi/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Omon/uo/gn/v20181126 

zpath is:  /dm4/naomi/zarr-test-Omon/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/v20181126/uo 

r3i1p1f1
ncpath is:  /dm5/naomi/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r3i1p1f1/Omon/uo/gn/v20181102 

zpath is:  /dm4/naomi/zarr-test-Omon/CMIP/BCC/BCC-CSM2-MR/historical/r3i1p1f1/v20181102/uo 

r2i1p1f1
ncpath is

OSError: no files to open

In [9]:
ds1 = xr.open_zarr('/dm4/naomi/zarr-test-Omon/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/v20181129/uo')